In [1]:
import numpy as np
import os
from PIL import Image
from sklearn.preprocessing import LabelBinarizer
import sys
import glob
import argparse
import matplotlib.pyplot as plt
import pickle as pkl
from keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from keras.models import Model, Sequential
from keras.layers import Dropout, Flatten,Dense, GlobalAveragePooling2D, LeakyReLU
from keras import optimizers
from tensorflow.keras.optimizers import Adam,SGD
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils.np_utils import to_categorical

from keras import backend as K
print('done')

done


In [2]:
Data_X = np.load() # path for Major training image data 
Data_Y = np.load() # path for Major training class data 

In [3]:
Data_Y = np.array(Data_Y)
print(np.unique(Data_Y))

['bcc' 'bkl' 'mel' 'nv' 'other']


In [4]:
Y_new = []
count_bcc = 0
count_bkl = 0
count_mel = 0
count_nv = 0
count_other = 0
for i in Data_Y:
  if i == 'bcc':
    Y_new.append(0)
    count_bcc = count_bcc + 1
  elif i == 'bkl':
    Y_new.append(1)
    count_bkl = count_bkl + 1
  elif i == 'mel':
    Y_new.append(2)
    count_mel = count_mel + 1
  elif i == 'nv':
    Y_new.append(3)
    count_nv = count_nv + 1
  elif i == 'other':
    Y_new.append(4)
    count_other = count_other + 1
print('bcc - ',count_bcc)
print('bkl - ',count_bkl)
print('mel - ', count_mel)
print('nv - ',count_nv)
print('other - ',count_other)
print(Y_new)
Y_new = np.array(Y_new)


bcc -  6646
bkl -  5248
mel -  6022
nv -  6705
other -  4272
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(Data_X, Y_new, test_size=0.15, random_state=69,stratify= Y_new)

In [6]:
print(X_train.shape)
print(Y_train.shape)
Y_train = to_categorical(Y_train, num_classes=5)
Y_test = to_categorical(Y_test, num_classes=5)

print('Train dataset shape',X_train.shape)
print('Test dataset shape',X_test.shape)

print(Y_train.shape)

(24559, 76, 76, 3)
(24559,)
Train dataset shape (24559, 76, 76, 3)
Test dataset shape (4334, 76, 76, 3)
(24559, 5)


In [7]:
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(input_shape=(76, 76,3), include_top=False, weights="imagenet")

In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [12]:
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

base_model.add(Dropout(0.1))
base_model = Sequential()
base_model.add(ResNet50(include_top=False, weights='imagenet', pooling='max'))
base_model.add(Dense(5, activation='softmax'))
base_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


In [13]:
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
start = time.time()

early_stopping_monitor = EarlyStopping(patience=100,monitor='val_accuracy')
model_checkpoint_callback = ModelCheckpoint(filepath='resnet.h5',
                                            save_weights_only=False,
                                            monitor='val_accuracy',
                                            mode='auto',
                                            save_best_only=True,
                                            verbose=1)

batch_size = 64
epochs = 250
optimizer = SGD(learning_rate=0.0001)
base_model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(zoom_range = 0.2, shear_range=0.2)

datagen.fit(X_train)

In [14]:
history=base_model.fit(datagen.flow(X_train,Y_train), epochs=epochs, batch_size=batch_size, shuffle=True, callbacks=[early_stopping_monitor,model_checkpoint_callback], validation_data=(X_test, Y_test))
end = time.time()
print("////////////////////////////Time Taken////////////////////////////////////////")
print(end-start)

Epoch 1/250
768/768 [==============================] - 67s 79ms/step - loss: 2.3674 - accuracy: 0.4263 - val_loss: 2.6475 - val_accuracy: 0.4052

Epoch 00001: val_accuracy improved from -inf to 0.40517, saving model to resnet.h5
Epoch 2/250
768/768 [==============================] - 58s 76ms/step - loss: 1.7014 - accuracy: 0.5209 - val_loss: 1.5842 - val_accuracy: 0.5399

Epoch 00002: val_accuracy improved from 0.40517 to 0.53992, saving model to resnet.h5
Epoch 3/250
768/768 [==============================] - 58s 76ms/step - loss: 1.4831 - accuracy: 0.5536 - val_loss: 1.4567 - val_accuracy: 0.5593

Epoch 00003: val_accuracy improved from 0.53992 to 0.55930, saving model to resnet.h5
Epoch 4/250
768/768 [==============================] - 58s 76ms/step - loss: 1.3272 - accuracy: 0.5803 - val_loss: 1.3265 - val_accuracy: 0.5805

Epoch 00004: val_accuracy improved from 0.55930 to 0.58053, saving model to resnet.h5
Epoch 5/250
768/768 [==============================] - 58s 75ms/step - loss


Epoch 00074: val_accuracy did not improve from 0.76973
Epoch 75/250
768/768 [==============================] - 59s 77ms/step - loss: 0.2497 - accuracy: 0.9093 - val_loss: 0.6532 - val_accuracy: 0.7711

Epoch 00075: val_accuracy improved from 0.76973 to 0.77111, saving model to resnet.h5
Epoch 76/250
768/768 [==============================] - 59s 77ms/step - loss: 0.2499 - accuracy: 0.9123 - val_loss: 0.6558 - val_accuracy: 0.7730

Epoch 00076: val_accuracy improved from 0.77111 to 0.77296, saving model to resnet.h5
Epoch 77/250
768/768 [==============================] - 59s 77ms/step - loss: 0.2406 - accuracy: 0.9140 - val_loss: 0.6527 - val_accuracy: 0.7773

Epoch 00077: val_accuracy improved from 0.77296 to 0.77734, saving model to resnet.h5
Epoch 78/250
768/768 [==============================] - 58s 76ms/step - loss: 0.2432 - accuracy: 0.9144 - val_loss: 0.6533 - val_accuracy: 0.7739

Epoch 00078: val_accuracy did not improve from 0.77734
Epoch 79/250
768/768 [=====================


Epoch 00113: val_accuracy did not improve from 0.79534
Epoch 114/250
768/768 [==============================] - 56s 73ms/step - loss: 0.1467 - accuracy: 0.9522 - val_loss: 0.6317 - val_accuracy: 0.7923

Epoch 00114: val_accuracy did not improve from 0.79534
Epoch 115/250
768/768 [==============================] - 57s 74ms/step - loss: 0.1451 - accuracy: 0.9537 - val_loss: 0.6297 - val_accuracy: 0.7930

Epoch 00115: val_accuracy did not improve from 0.79534
Epoch 116/250
768/768 [==============================] - 57s 75ms/step - loss: 0.1428 - accuracy: 0.9547 - val_loss: 0.6295 - val_accuracy: 0.7951

Epoch 00116: val_accuracy did not improve from 0.79534
Epoch 117/250
768/768 [==============================] - 57s 74ms/step - loss: 0.1407 - accuracy: 0.9548 - val_loss: 0.6283 - val_accuracy: 0.7953

Epoch 00117: val_accuracy did not improve from 0.79534
Epoch 118/250
768/768 [==============================] - 56s 73ms/step - loss: 0.1384 - accuracy: 0.9551 - val_loss: 0.6288 - val_ac

KeyboardInterrupt: 